In [ ]:
## Reference
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
# https://machinelearningmastery.com/implement-resampling-methods-scratch-python/

#http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.DOCUMENTATION

In [1]:
## LIBRARIES

import numpy as np
import pandas as pd
import math
import random
from collections import Counter


In [2]:
## LOAD DATA
data = pd.read_csv('spambase_data.txt', sep = ",",header=None)

In [3]:
##K-FOLDS

k = 5
fold_size = math.ceil(len(data) / k)
dataset_split = list()
ind_count = 0
for i in range(k):
    #print(i)
    dataset_split.append(data.iloc[ind_count:ind_count+fold_size,:])
    ind_count = ind_count + fold_size + 1
    
def create_train_test(train_ind, test_ind):
    train = pd.DataFrame()
    test = pd.DataFrame()
    for i in train_ind:
        train = train.append(dataset_split[i])
    #for i in test_ind:
    test = test.append(dataset_split[i])
    return(train, test)

In [21]:
## TERMINAL NODES

# Create a terminal node value - Gets mean value of group
def to_terminal(group):
    outcomes = group.iloc[:,57]
    if len(outcomes) > 0:
        return (Counter(outcomes).most_common(1)[0][0])
    else:
        return(0)
    #return max(set(outcomes), key=outcomes.count)
    #return (Counter(outcomes).most_common(1)[0][0])

In [5]:
##ENTROPY
def get_entropy(groups):
    entropy_node = 0  #Initialize Entropy
      
    entropy = 0
    #if groups is actually just 1 df
    if len(groups) > 2:
        values = groups[57].unique()
        for value in values:
            fraction = groups.iloc[:,57].value_counts()[value]/len(groups.iloc[:,57])  
            entropy_node += -fraction*np.log2(fraction)
        entropy = entropy_node
    #if groups is (df,df)
    else:
        n_instances = float(sum([len(group) for group in groups]))
        for g in groups:
            values = g[57].unique()
            #print(g)
            for value in values:
                fraction = g.iloc[:,57].value_counts()[value]/len(g.iloc[:,57])  
                entropy_node += -fraction*np.log2(fraction)
            entropy += entropy_node * (len(g)/n_instances)
    return(entropy)
        
def information_gain(ent_orig, ent_split):
    return(ent_orig - ent_split)

In [6]:
#BUILD TREE

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    #print(root)
    split(root, max_depth, min_size, 1)
    return root

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
#     if not left or not right:
#         node['left'] = node['right'] = to_terminal(left + right)
#         return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['col']+1), node['val'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [7]:
## SPLITS

# Split a dataset based on an column and an attribute value
# RETURNS 2 dataframes
def test_split(col, value, dataset):
    left = dataset.loc[dataset[col] < value]
    right = dataset.loc[dataset[col] >= value]
    return left, right

# Select the best split point for a dataset
def get_split(dataset):
    b_col, b_val, b_ig, b_groups = np.inf, np.inf, None, None
    
    #iterate rows
    for col in dataset.iloc[:,:-1]:
        colmax = dataset[col].max()
        colmin = dataset[col].min()
        test_values = np.linspace(colmin,colmax,100)
        
        for i in test_values:
            groups = test_split(col, i, dataset)
            #print(groups)
            ig = information_gain(get_entropy(dataset), get_entropy(groups))
            #print(ig)
            if b_ig is None:
                b_ig = ig
                b_col = col
                b_groups = groups
                b_val = i
            elif ig > b_ig:
                b_ig = ig
                b_col = col
                b_groups = groups
                b_val = i
                
    return {'col':b_col, 'val':b_val, 'groups':b_groups}

In [13]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['col']] < node['val']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
        
# Classification and Regression Tree Algorithm returns MSE
def decision_tree(tree, test):
    #tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for i in range(len(test[0])):
        prediction = predict(tree, test.iloc[i,:])
        predictions.append(prediction)
    mse = 0
    for i in range(len(predictions)):
        mse += ((predictions[i] - test.iloc[i,57]) ** 2) / len(predictions)
    return(mse)

In [14]:
def init_dec_tree(k, train_ind, test_ind):
    print("Epoch " + str(k+1))
    train,test = create_train_test(train_ind, test_ind)
    tree = build_tree(train, 5, 10)

    train_error = decision_tree(tree,train)
    print('Train MSE: ' + str(train_error))
    test_error = decision_tree(tree,test)
    print('Test MSE: ' + str(test_error))
    return(test_error)

In [22]:
#LOOCV
total_mse = 0

for i in range(k):
    test_ind = i
    train_ind = [x for x in range(k) if x!=i]
    #train,test = create_train_test(train_ind, test_ind)
    total_mse += init_dec_tree(i, train_ind, test_ind)
print("Average Test MSE: " + str(total_mse/k))

Epoch 1
Train MSE: 0.0696409140369967
Test MSE: 0.03285870755750272
Epoch 2
Train MSE: 0.07100108813928176
Test MSE: 0.04819277108433732
Epoch 3
Train MSE: 0.10418933623503725
Test MSE: 0.06243154435925516
Epoch 4
Train MSE: 0.10799782372143542
Test MSE: 0.12924424972617776
Epoch 5
Train MSE: 0.15960912052117268
Test MSE: 0.3463626492942455
Average Test MSE: 0.1238179844043037


In [ ]:

train,test = create_train_test(train_ind, test_ind)

In [ ]:
train_error, train_tree = decision_tree(train, train, 3, 5)

In [20]:
outcomes = []
Counter(outcomes).most_common(1)[0][0]

IndexError: list index out of range